In [2]:
import torch

from dataset import *
from model import *
from utils import *

from collections import OrderedDict

In [4]:
data_path = '/data/Cityscapes'
save_path = 'checkpoints/same_epoch_45.pth'

padding = 'same' if save_path.split(sep='/')[1].startswith('same') else 'valid'
dataset = Cityscapes(root=data_path, split='val')
model = UNET(n_classes=34, padding=padding)

save_state = torch.load(save_path)
weights = save_state['state_dict']
history = save_state['history']

model.load_state_dict(weights)

take 2 worked


In [ ]:
%matplotlib notebook
i = np.random.randint(low=0, high=len(dataset)-1)

print("Plotting image {} from the '{}' dataset.".format(i, dataset.split))
test(i, dataset=dataset, model=model)
plt.savefig('fig.pdf')
# Image 377 from 'val' dataset

In [ ]:
%matplotlib notebook

plt.figure(1, figsize=(8, 3))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='training set')
plt.plot(history['val_loss'], label='validation set')
plt.legend(prop={'size':12})
plt.grid()
plt.title('Loss', fontsize=20)
plt.xlabel('# epochs', fontsize=15)

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='training set')
plt.plot(history['val_acc'], label='validation set')
plt.legend(prop={'size':12})
plt.grid()
plt.title('Accuracy', fontsize=20)
plt.xlabel('# epochs', fontsize=15)
plt.ylim([0, 1])
plt.tight_layout()
#plt.savefig('metrics.png')



In [ ]:
%matplotlib notebook
class_ids = dataset.class_ids
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True)

for i in [7, 8, 10, 11, 12]:
    ax1.plot(history['val_iou'][:, i], label=class_ids[i])
ax1.legend(prop={'size': 8}, loc='right')
ax1.grid()
ax1.set_title('Surfaces')

for i in [26, 27, 28, 29, 30, 31, 32]:
    ax2.plot(history['val_iou'][:, i], label=class_ids[i])
ax2.legend(prop={'size':8}, loc='right')
ax2.grid()
ax2.set_title('Vehicles')
ax2.set_ylim([-.01, 1.])

for i in [21, 22, 23, 24, 33]:
    ax3.plot(history['val_iou'][:, i], label=class_ids[i])
ax3.legend(prop={'size':8}, loc='upper right')
ax3.grid()
ax3.set_title('Nature / People')
ax3.set_ylim([-.01, 1.])
ax3.set_xlabel('# epochs')

for i in [19, 20, 17, 13]:
    ax4.plot(history['val_iou'][:, i], label=class_ids[i])
ax4.legend(prop={'size':8})
ax4.grid()
ax4.set_title('Traffic / Obstacles')
ax4.set_ylim([-.01, 1.])
ax4.set_xlabel('# epochs')

plt.subplots_adjust(wspace=.05)
plt.tight_layout()
#plt.savefig('classious.png')


In [ ]:
model_same = model
history_same = history

save_path = 'checkpoints/epoch_45.pth'

padding = 'same' if save_path.split(sep='/')[1].startswith('same') else 'valid'
dataset = Cityscapes(root=data_path, split='val')
model = UNET(n_classes=34, padding=padding)

save_state = torch.load(save_path)
weights = save_state['state_dict']
history = save_state['history']
try:
    print('>> Loading model weights from: {}'.format(save_path))
    model.load_state_dict(weights)
except:
    # If the model was trained on parallel GPUs, it needs this jenky workaround to load it onto CPU
    print('>> Model was trained on parallel GPUs. Loading weights onto CPU for evaluation.')
    new_weights = OrderedDict()
    for k, v in weights.items():
        name = k[7:]
        new_weights[name] = v
    model.load_state_dict(new_weights)
    
model_valid = model
history_valid = history

In [ ]:
%matplotlib notebook

plt.figure(1)
plt.plot(history_same['val_acc'], label="'same' padding")
plt.plot(history_valid['val_acc'], label="'valid' padding", linestyle='dashed')
plt.legend(prop={'size':16})
plt.grid()
plt.ylim([0, 1.])
plt.title('Accuracy', fontsize=18)
plt.xlabel('# epochs', fontsize=18)
plt.savefig('padding.png')